https://www.kaggle.com/laxmikantnishad/covid-19-2-para

In [1]:
is_local=True

In [2]:
if not is_local:
    !conda install '/kaggle/input/pydicom-conda-helper/libjpeg-turbo-2.1.0-h7f98852_0.tar.bz2' -c conda-forge -y
    !conda install '/kaggle/input/pydicom-conda-helper/libgcc-ng-9.3.0-h2828fa1_19.tar.bz2' -c conda-forge -y
    !conda install '/kaggle/input/pydicom-conda-helper/gdcm-2.8.9-py37h500ead1_1.tar.bz2' -c conda-forge -y
    !conda install '/kaggle/input/pydicom-conda-helper/conda-4.10.1-py37h89c1867_0.tar.bz2' -c conda-forge -y
    !conda install '/kaggle/input/pydicom-conda-helper/certifi-2020.12.5-py37h89c1867_1.tar.bz2' -c conda-forge -y
    !conda install '/kaggle/input/pydicom-conda-helper/openssl-1.1.1k-h7f98852_0.tar.bz2' -c conda-forge -y

In [3]:
import os

from PIL import Image
import pandas as pd
import numpy as np
from tqdm.auto import tqdm

（1）dcm to .png

In [4]:
def read_xray(path, voi_lut = True, fix_monochrome = True):
    import pydicom
    from pydicom.pixel_data_handlers.util import apply_voi_lut
    # Original from: https://www.kaggle.com/raddar/convert-dicom-to-np-array-the-correct-way
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data
        

In [5]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [6]:
def prepare_test_images():
    split = 'test'
    save_dir = f'./{split}/'

    os.makedirs(save_dir, exist_ok=True)

    save_dir = f'./{split}/study/'
    os.makedirs(save_dir, exist_ok=True)

    for dirname, _, filenames in tqdm(os.walk(f'../input/siim-covid19-detection/{split}')):
        for file in filenames:
            # set keep_ratio=True to have original aspect ratio
            xray = read_xray(os.path.join(dirname, file))
            im = resize(xray, size=512)  
            study = dirname.split('/')[-2] + '_study.png'
            #study = dirname.split('/')[-2] + '_study'
            im.save(os.path.join(save_dir, study))

In [7]:
import sys; 

package_paths = [
    '../input/image-fmix/FMix-master'                            #FMix是一种数据增强方法（最近比较火的一种）
]

if not is_local:
    package_paths.append('../input/pytorch-image-models/pytorch-image-models-master') #导入pytorch模型

for pth in package_paths:
    sys.path.append(pth)

In [8]:
from glob import glob
import cv2
from skimage import io
import torch
from torch import nn
import os
from datetime import datetime
import time
import random
import torchvision
from torchvision import transforms
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.cuda.amp import autocast, GradScaler
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

import timm
import sklearn
import warnings
import joblib
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
import warnings
from scipy.ndimage.interpolation import zoom

from fmix import sample_mask, make_low_freq_image, binarise_mask
from sklearn.model_selection import GroupKFold, StratifiedKFold

In [9]:
# COMPETITION_NAME = "siimcovid19-512-img-png-600-study-png"
# load_dir = f"/kaggle/input/{COMPETITION_NAME}/"

In [10]:
# df.rename(columns={'Negative for Pneumonia':'0','Typical Appearance':'1',"Indeterminate Appearance":'2',
#                    "Atypical Appearance":"3"}, inplace=True)
# # one-hot
# labels = []
# def get_label(row):
#     for c in df.columns:
#         if row[c]==1:
#             labels.append(int(c))
# df.apply(get_label, axis=1)
# print("label modified")

# labels = {'label':labels}
# study_label = pd.DataFrame(labels)
# train_study = pd.concat([df, study_label], axis = 1)
# #print(train_study)
# del train_study ['0'];del train_study ['1'];del train_study ['2'];del train_study ['3']
# train_study

In [11]:
# CFG = {
#     'fold_num': 5,
#     'seed': 719,
#     'model_arch': 'tf_efficientnet_b4_ns',
#     'img_size': 512,
#     'epochs': 10,
#     'train_bs': 32,
#     'valid_bs': 32,
#     'lr': 1e-4,
#     'num_workers': 4,
#     'accum_iter': 1, # suppoprt to do batch accumulation for backprop with effectively larger batch size
#     'verbose_step': 1,
#     'device': 'cuda:0',
#     'tta': 3,
#     'used_epochs': [53,55,56,59],
#     'weights': [1,1,1,1]
# }

In [12]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    #print(im_rgb)
    return im_rgb

#img = get_img('../input/siim-covid19-detection/test/00188a671292/3eb5a506ccf3/3dcdfc352a06.dcm')
#plt.imshow(img)
#plt.show()

In [13]:
class CassavaDataset(Dataset):
    def __init__(
        self, df, data_root, transforms=None, output_label=True
    ):
        
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.id_col_name = 'study_id'
        self.transforms = transforms
        self.data_root = data_root
        self.output_label = output_label
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index: int):
        
        # get labels
        if self.output_label:
            target = self.df.iloc[index]['class_id']
        
        tempstr = self.df.iloc[index][self.id_col_name][-3:]
        if tempstr == 'png':
            path = "{}/{}".format(self.data_root, self.df.iloc[index][self.id_col_name])
        else:
            path = "{}/{}".format(self.data_root, self.df.iloc[index][self.id_col_name])+'.png'
        
        img  = get_img(path)
        
        if self.transforms:
            img = self.transforms(image=img)['image']
            
        # do label smoothing
        if self.output_label == True:
            return img, target
        else:
            return img

In [14]:
from albumentations import (
    HorizontalFlip, VerticalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine, RandomResizedCrop,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, Normalize, Cutout, CoarseDropout, ShiftScaleRotate, CenterCrop, Resize
)

from albumentations.pytorch import ToTensorV2

def get_train_transforms():
    return Compose([
            RandomResizedCrop(cfg['img_size'], cfg['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            CoarseDropout(p=0.5),
            Cutout(p=0.5),
            ToTensorV2(p=1.0),
        ], p=1.)
  
        
def get_valid_transforms():
    return Compose([
            CenterCrop(cfg['img_size'], cfg['img_size'], p=1.),
            Resize(cfg['img_size'], cfg['img_size']),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

def get_inference_transforms():
    return Compose([
            RandomResizedCrop(cfg['img_size'], cfg['img_size']),
            Transpose(p=0.5),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0),
        ], p=1.)

In [15]:
class CassvaImgClassifier(nn.Module):
    def __init__(self, model_arch, n_class, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [16]:
def inference_one_epoch(model, data_loader, device):
    model.eval()

    image_preds_all = []
    
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()
        
        image_preds = model(imgs)   #output = model(input)
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
        
    
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all

In [17]:
def create_sub_df(testid, tst_preds):
    sub_df = pd.read_csv('../input/siim-covid19-detection/sample_submission.csv')
    for i in range(len(test)):
        for j in range(len(test)):
            a = test.loc[i,'id'].split('.')[0]
            b = sub_df.loc[j,'id']
            if a==b:
                negative, typical, indeterminate, atypical = str(tst_preds[i][0]),str(tst_preds[i][1]),str(tst_preds[i][2]),str(tst_preds[i][3]),
                sub_df.loc[j,'PredictionString'] = f'negative {negative} 0 0 1 1 typical {typical} 0 0 1 1 indeterminate {indeterminate} 0 0 1 1 atypical {atypical} 0 0 1 1'
    return sub_df

In [18]:
def inference(cfg, is_local):
    if is_local:
        print('Inference on Validation dataset')
    else:
        print('Inference on Test dataset')

    seed_everything(cfg['seed'])
    # folds = StratifiedKFold(n_splits=CFG['fold_num']).split(np.arange(train_study.shape[0]), train_study.label.values)

    if is_local:
        model_dir = f'../models/{cfg["exp_id"]}/'
        train_img_path = '../input/siimcovid19-512-img-png-600-study-png/study/'
        test_img_path = './test/study/'
        train_csv_path = f'../input/siim_covid19_train/{cfg["train_csv_name"]}'
        train = pd.read_csv(train_csv_path)
    else:
        model_dir = f'../input/siim_covid19_studyclass_models/{cfg["exp_id"]}/'
        train_img_path = '../input/siimcovid19-512-img-png-600-study-png/study/'
        test_img_path = './test/study/'
        train_csv_path = f'../input/siim_covid19_train/{cfg["train_csv_name"]}'
        prepare_test_images()
    
    accs = []
    # for fold, (trn_idx, val_idx) in enumerate(folds):
    for idx, fold in enumerate(range(cfg['fold_num'])):
        # train_df = train[train['fold'] != fold]
        valid_df = train[train['fold'] == fold]
        # we'll train fold 0 first
#         if fold > 0:
#             break 

        print('Inference fold {} started'.format(fold))

        if is_local:
            # valid_ = train_study.loc[val_idx,:].reset_index(drop=True)
            #print('valid_',valid_)
            valid_ds = CassavaDataset(valid_df, train_img_path, transforms=get_inference_transforms(), output_label=False)
            val_loader = torch.utils.data.DataLoader(
                valid_ds, 
                batch_size=cfg['valid_bs'],
                num_workers=cfg['num_workers'],
                shuffle=False,
                pin_memory=False,
            )
            data_loader = val_loader
        else:
            test = pd.DataFrame()
            test['id'] = list(os.listdir(test_img_path))
            #print('test',test)
            test_ds = CassavaDataset(test, test_img_path, transforms=get_inference_transforms(), output_label=False)
            #print('test_ds',test)
            
            tst_loader = torch.utils.data.DataLoader(
                test_ds, 
                batch_size=cfg['valid_bs'],
                num_workers=cfg['num_workers'],
                shuffle=False,
                pin_memory=False,
            )
            data_loader = tst_loader

        device = torch.device(cfg['device'])
        print(f'Num Class: {valid_df.class_id.nunique()}')
        model = CassvaImgClassifier(cfg['model_arch'], valid_df.class_id.nunique()).to(device)
        
        # val_preds = []
        # tst_preds = []
        preds = []
        
        #for epoch in range(CFG['epochs']-3):
        # for i, epoch in enumerate(cfg['used_epochs']):    
        model_path = f'{model_dir}best_acc_{cfg["model_arch"]}_fold{fold}.pt'
        print(f'Loading {model_path}')
        model.load_state_dict(torch.load(model_path))
        
        with torch.no_grad():
            for _ in range(cfg['tta']):
                preds += [cfg['weights'][0]/sum(cfg['weights'])/cfg['tta']*inference_one_epoch(model=model, data_loader=data_loader, device=device)]

        preds = np.mean(preds, axis=0) 
        if is_local:
            acc = (valid_df.class_id.values==np.argmax(preds, axis=1)).mean()
            print('fold {} validation loss = {:.5f}'.format(fold, log_loss(valid_df.class_id.values, preds)))
            print('fold {} validation accuracy = {:.5f}'.format(fold, acc))
            accs.append(acc)
        
        del model
        torch.cuda.empty_cache()
    
    if not is_local:
        sumfile = create_sub_df(test, preds)
        sumfile.to_csv('./submission.csv',index=False)
        !rm -r ./test/study/
    else:
        mean_acc = (sum(accs)/cfg["fold_num"])*100.0
        print(f'Mean Accuracy {mean_acc}')

# 4 Main

In [19]:
if __name__ == '__main__':
    cfg = {
        'exp_id': 'exp1',
        'train_csv_name':'train_classification_fold_v1.csv',
        'fold_num': 5,
        'seed': 719,
        #'model_arch': 'tf_efficientnet_b7',
        'model_arch': 'tf_efficientnet_b4_ns',
        'img_size': 512,
        'epochs': 100,
        'train_bs': 14,
        'valid_bs': 32,
        'T_0': 10,
        'lr': 1e-6,
        'min_lr': 1e-6,
        'weight_decay':1e-6,
        'num_workers': 4,
        'accum_iter': 2, # suppoprt to do batch accumulation for backprop with effectively larger batch size
        'verbose_step': 1,
        'device': 'cuda:0',
        'tta': 3,
        'used_epochs': [53,55,56,59],
        'weights': [1,1,1,1]
    }

    inference(cfg, is_local)

Inference on Validation dataset
Inference fold 0 started
Num Class: 4
Loading ../models/exp1/best_acc_tf_efficientnet_b4_ns_fold0.pt


100%|██████████| 38/38 [00:11<00:00,  3.44it/s]


fold 0 validation loss = 0.94210
fold 0 validation accuracy = 0.64410
Inference fold 1 started
Num Class: 4
Loading ../models/exp1/best_acc_tf_efficientnet_b4_ns_fold1.pt


100%|██████████| 38/38 [00:10<00:00,  3.52it/s]


fold 1 validation loss = 0.95327
fold 1 validation accuracy = 0.63501
Inference fold 2 started
Num Class: 4
Loading ../models/exp1/best_acc_tf_efficientnet_b4_ns_fold2.pt


...

fold 2 validation loss = 0.93656
fold 2 validation accuracy = 0.64410
Inference fold 3 started
Num Class: 4
Loading ../models/exp1/best_acc_tf_efficientnet_b4_ns_fold3.pt


100%|██████████| 38/38 [00:10<00:00,  3.54it/s]


fold 3 validation loss = 0.92083
fold 3 validation accuracy = 0.65400
Inference fold 4 started
Num Class: 4
Loading ../models/exp1/best_acc_tf_efficientnet_b4_ns_fold4.pt


100%|██████████| 38/38 [00:10<00:00,  3.53it/s]

fold 4 validation loss = 0.93776
fold 4 validation accuracy = 0.63554
Mean Accuracy 64.25492216664051


In [20]:
# fold 0 validation loss = 0.94095
# fold 0 validation accuracy = 0.64162